In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd

In [7]:
# Geckodriver yolu
geckodriver_path = "./geckodriver"

# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)


# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)
# Hedef web sayfasına git

macKolikUrl="https://www.mackolik.com/futbol/canli-sonuclar"
driver.get(macKolikUrl)

#time.sleep(5)
wait = WebDriverWait(driver, 10)
cookies=wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]')))
cookies.click()

clock=wait.until(EC.element_to_be_clickable((By.XPATH, "//span[.='Saat']")))
clock.click()

dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))

# Click on the element
dateIcon.click()
butunTakvim = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]')))
html_contentTakvim = butunTakvim.get_attribute('outerHTML')
# print(html_contentTakvim)



# BeautifulSoup ile HTML içeriğini parse et
soup = BeautifulSoup(html_contentTakvim, 'html.parser')

# Günü ve Yılı al
day = int(soup.find('td', {'class': 'widget-datepicker__calendar-body-cell--selected-day'}).text.strip())
year = int(soup.find('div', {'class': 'widget-datepicker__selector--year'}).find('div', {'class': 'widget-datepicker__value'}).text.strip())

#print(f"day: {day}, year: {year}")

current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
previous_year_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='widget-dateslider__datepicker widget-dateslider__datepicker--open']/div/div/div[2]/div[1]")))

# Hedef yılı belirle (örneğin, 2020)
target_year = 2020

# Hedef yıla kadar önceki yıl düğmesine tıkla
while current_year > target_year:
    #print(str(target_year)+ "target year")
    previous_year_button.click()
    #time.sleep(1)
    current_year -= 1
    #print(current_year)
    # Yeni yılı kontrol et
    current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
    #print(current_year)

    
# calc of months
previous_month_button= wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[1]")))
#wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
month_str = element.text.strip()[:3]

#print(month_str)
# Map the month abbreviation to a numeric value
month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
#print(month_str+"month_str")
month_numeric = month_mapping.get(month_str)

target_month=12
month_numeric = month_mapping.get(month_str)


# Loop until you reach the target month
while month_numeric != target_month:
    #print(str(month_numeric)+"month_numeric")
    #print(str(target_month)+"target_month")

    
    previous_month_button.click()
    #time.sleep(1)
    
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
    month_str = element.text.strip()[:3]
    #print(month_str+"month_str")
    month_numeric = month_mapping.get(month_str)
    #print(month_numeric)
    #print(target_month)
    
target_day=19
day_element = driver.find_element(By.XPATH, f"//td[@class='widget-datepicker__calendar-body-cell' and text()='{target_day}']")
#time.sleep(19)
day_element.click()
time.sleep(3)


container_xpath = "//div[@class='match-row__match-content']"  #(//div[@class='match-row__match-content'])[1]

# Container elementini bul
allLinksElements = driver.find_elements(By.XPATH, container_xpath)

df_all = pd.DataFrame()
count = 0

while True:
    try:
        # Container elementini bul
        allLinksElements = driver.find_elements(By.XPATH, container_xpath)

        # If all links are processed, break out of the loop
        if not allLinksElements:
            break

        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=1):
            try:
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                elementx = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, dynamic_xpath)))
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                element.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                time.sleep(1)
                # Perform actions in the new tab
                iddiaButton = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[1]/div[2]/div/a[3]")))

                tx = iddiaButton.get_attribute('class')
                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    macScorboard = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']")))
                    macDetailsData = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div")))
                    macScorboard.text
                    macDetailsData.text
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    time.sleep(3)
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(3)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                count += 1
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue


(try Worked... 1)
(went new page... 1)
Button is not disabled. Proceeding...
(gone new page closed.. 1)
(try Worked... 2)
(went new page... 2)
Button is not disabled. Proceeding...
(gone new page closed.. 2)
(try Worked... 3)
(went new page... 3)
Button is disabled. Skipping...
(try Worked... 4)
(went new page... 4)
Button is disabled. Skipping...
(try Worked... 5)
(went new page... 5)
Button is not disabled. Proceeding...
(gone new page closed.. 5)
(try Worked... 6)
(went new page... 6)
Button is not disabled. Proceeding...
(gone new page closed.. 6)
(try Worked... 7)
(went new page... 7)
Button is not disabled. Proceeding...
(gone new page closed.. 7)
(try Worked... 8)
(went new page... 8)
Button is not disabled. Proceeding...
(gone new page closed.. 8)
(try Worked... 9)
(went new page... 9)
Button is not disabled. Proceeding...
(gone new page closed.. 9)
(try Worked... 10)
(went new page... 10)
Button is not disabled. Proceeding...
(gone new page closed.. 10)
(try Worked... 11)
(wen

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16
